### General Security Analysis for Face Template Protection Methods from Cryptographic Hash Functions

- Seunghun Paik, Minsu Kim, Sunpill Kim, and Jae Hong Seo

- Department of Mathematics & Research Institute for Natural Sciences, Hanyang University

- Submitted to IEEE Transactions on Information Forensics and Security.

#### This notebook demonstrates the proposed security estimator for CtH-based BTPs

In [ ]:
### Configs for models & datasets
from config import cfg
device = "cuda:1"
batch_size = 256

#### Step 1. Load Benchmark Datasets

Throughout this notebook, we will utilize LFW only.

In [ ]:
import os
from feat_tools import load_bin

lfw_dir = os.path.join(cfg.img_dir, "lfw.bin")
lfw = load_bin(lfw_dir, (112,112))

#### Step 2. Benchmarking BTPs

Throughout this notebook, we will use `cfg.model3` as a backbone.

In [ ]:
from feat_tools import feat_ext
from backbones import get_backbone
from benchmark import benchmark_plain, benchmark_IM, benchmark_SF
from BTPs import IronMask, SigFeat

model = get_backbone(cfg.model3, device)
lfw_feats = feat_ext(lfw, model, batch_size, device).cpu()

###### Plain

In [ ]:
thx, ACC, TAR, FAR = benchmark_plain(lfw_feats, target_far=0)

print("Benchmark, Plain")
print(f"TAR: {TAR.item() * 100:.2f}%")
print(f"FAR: {FAR.item() * 100:.2f}%")
print(f"ACC: {ACC.item() * 100:.2f}%")
print(f"THX: {thx.item():.4f}")

###### IronMask

In [ ]:
IM = IronMask(512, 8)

TAR, FAR, ACC = benchmark_IM(lfw_feats, IM, batch_size)
print("Benchmark, IronMask-64bit")
print(f"TAR: {TAR.item() * 100:.2f}%")
print(f"FAR: {FAR.item() * 100:.2f}%")
print(f"ACC: {ACC.item() * 100:.2f}%")

print()

IM = IronMask(512, 16)

TAR, FAR, ACC = benchmark_IM(lfw_feats, IM, batch_size)
print("Benchmark, IronMask-115bit")
print(f"TAR: {TAR.item() * 100:.2f}%")
print(f"FAR: {FAR.item() * 100:.2f}%")
print(f"ACC: {ACC.item() * 100:.2f}%")

###### Mohan et al. (BCH)

In [ ]:
Mohan = SigFeat(512, 127, 64)

TAR, FAR, ACC = benchmark_SF(lfw_feats, Mohan, batch_size=500)
print("Benchmark, Mohan et al.-64bit (BCH)")
print(f"TAR: {TAR.item() * 100:.2f}%")
print(f"FAR: {FAR.item() * 100:.2f}%")
print(f"ACC: {ACC.item() * 100:.2f}%")

print()

Mohan = SigFeat(512, 255, 115)

TAR, FAR, ACC = benchmark_SF(lfw_feats, Mohan, batch_size=500)
print("Benchmark, Mohan et al.-115bit (BCH)")
print(f"TAR: {TAR.item() * 100:.2f}%")
print(f"FAR: {FAR.item() * 100:.2f}%")
print(f"ACC: {ACC.item() * 100:.2f}%")


###### Mohan et al. (Reed-Solomon)

In [ ]:
Mohan = SigFeat(512, 127, 64, mode = "RS")

TAR, FAR, ACC = benchmark_SF(lfw_feats, Mohan, batch_size=500)
print("Benchmark, Mohan et al.-64bit (Reed-Solomon)")
print(f"TAR: {TAR.item() * 100:.2f}%")
print(f"FAR: {FAR.item() * 100:.2f}%")
print(f"ACC: {ACC.item() * 100:.2f}%")

print()

Mohan = SigFeat(512, 255, 115, mode = "RS")

TAR, FAR, ACC = benchmark_SF(lfw_feats, Mohan, batch_size=500)
print("Benchmark, Mohan et al.-115bit (Reed-Solomon)")
print(f"TAR: {TAR.item() * 100:.2f}%")
print(f"FAR: {FAR.item() * 100:.2f}%")
print(f"ACC: {ACC.item() * 100:.2f}%")

#### Step 3. Security Estimator

### IronMask

In [ ]:
from estimator import estimator_IM

#### Caveat: The estimator is a probabilistic algorithm; it may gives some random results

In [ ]:
# 64-bit security parameter
LB, UB = estimator_IM(512, 8)

print(f"Lower Bound: {-UB}-bits")
print(f"Upper Bound: {-LB}-bits")

In [ ]:
# 115-bit security parameter
LB, UB = estimator_IM(512, 16)

print(f"Lower Bound: {-UB}-bits")
print(f"Upper Bound: {-LB}-bits")

### Mohan et al. (BCH code)

In [ ]:
from estimator import estimator_SF

In [ ]:
# 64-bit security parameter
Mohan = SigFeat(512, 127, 64)
LB, UB = estimator_SF(Mohan)

print(f"Lower Bound: {-UB}-bits")
print(f"Upper Bound: {-LB}-bits")

In [ ]:
# 115-bit security parameter
Mohan = SigFeat(512, 255, 115)
LB, UB = estimator_SF(Mohan)

print(f"Lower Bound: {-UB}-bits")
print(f"Upper Bound: {-LB}-bits")

### Mohan et al. (RS code)

In [ ]:
# 64-bit security parameter
Mohan = SigFeat(512, 127, 64, mode = "RS")
LB, UB = estimator_SF(Mohan)

print(f"Lower Bound: {-UB}-bits")
print(f"Upper Bound: {-LB}-bits")

In [ ]:
# 115-bit security parameter
Mohan = SigFeat(512, 255, 115, mode = "RS")
LB, UB = estimator_SF(Mohan)

print(f"Lower Bound: {-UB}-bits")
print(f"Upper Bound: {-LB}-bits")